# LangGraph Agentic RAG - Example Usage

This notebook demonstrates how to use the LangGraph Agentic RAG system.

In [ ]:
# Setup imports
import sys
sys.path.append('..')

import asyncio
from src.agents.main_agent import AgentRunner
from src.ingestion.pipeline import IngestionPipeline

## 1. Ingest Sample Documents

First, let's ingest some documents to build our knowledge graph.

In [ ]:
# Create sample document
sample_doc = """
SpyroSolutions is a leading technology company founded in 2015 by Jane Smith and John Doe.
The company specializes in cloud computing solutions and has offices in San Francisco and New York.

Their flagship product, SpyroCloud, is used by over 1000 enterprise customers including TechCorp
and FinanceHub. SpyroCloud provides advanced data analytics and machine learning capabilities.

In 2023, SpyroSolutions acquired DataStream Inc. for $50 million to enhance their real-time
processing capabilities. The company reported $200 million in annual recurring revenue (ARR)
with a 95% customer retention rate.
"""

# Save to file
with open('../data/sample_company.txt', 'w') as f:
    f.write(sample_doc)

print("Sample document created")

In [ ]:
# Initialize ingestion pipeline
pipeline = IngestionPipeline()

# Ingest the document
result = await pipeline.ingest_file('../data/sample_company.txt')
print(f"Ingestion result: {result}")

# Close pipeline
pipeline.close()

## 2. Query the System

Now let's test different types of queries to see how the agent routes and responds.

In [ ]:
# Initialize the agent runner
agent = AgentRunner()

### Simple Greeting

In [ ]:
# Test a simple greeting
response = await agent.run("Hello! How are you?")
print(f"Answer: {response['answer']}")
print(f"Route: {response['metadata'].get('route')}")

### Fact Lookup Query

In [ ]:
# Test a fact lookup query
response = await agent.run("What is SpyroSolutions?")
print(f"Answer: {response['answer']}")
print(f"Route: {response['metadata'].get('route')}")
print(f"Tools used: {response['metadata'].get('tools_used')}")

### Hybrid Search Query

In [ ]:
# Test a hybrid search query with specific names
response = await agent.run("Tell me about SpyroCloud and its customers like TechCorp")
print(f"Answer: {response['answer']}")
print(f"Route: {response['metadata'].get('route')}")
print(f"Tools used: {response['metadata'].get('tools_used')}")

### Relational Query

In [ ]:
# Test a relational query
response = await agent.run("Who founded SpyroSolutions and what companies did they acquire?")
print(f"Answer: {response['answer']}")
print(f"Route: {response['metadata'].get('route')}")
print(f"Tools used: {response['metadata'].get('tools_used')}")

## 3. Conversation Continuity

Test the conversation persistence feature.

In [ ]:
# Start a conversation
response1 = await agent.run("What is the ARR of SpyroSolutions?")
thread_id = response1['thread_id']
print(f"Answer 1: {response1['answer']}")
print(f"Thread ID: {thread_id}")

In [ ]:
# Continue the conversation
response2 = await agent.continue_conversation(
    "How does that compare to their acquisition cost?",
    thread_id=thread_id
)
print(f"Answer 2: {response2['answer']}")

## 4. Error Handling

Test how the system handles errors and ambiguous queries.

In [ ]:
# Test an ambiguous query
response = await agent.run("Tell me about that thing")
print(f"Answer: {response['answer']}")
print(f"Route: {response['metadata'].get('route')}")

## 5. Direct Tool Testing

You can also test the retrieval tools directly.

In [ ]:
from src.tools.search_tools import vector_search_tool, graph_query_tool

# Test vector search
vector_results = await vector_search_tool.ainvoke({"query": "cloud computing solutions", "k": 3})
print("Vector Search Results:")
print(vector_results[:500] + "..." if len(vector_results) > 500 else vector_results)

In [ ]:
# Test graph query
graph_results = await graph_query_tool.ainvoke({"query": "What companies are customers of SpyroSolutions?"})
print("Graph Query Results:")
print(graph_results[:500] + "..." if len(graph_results) > 500 else graph_results)